In [1]:
from sympy import Symbol, Add, Mul, simplify, cancel
from sympy.core.function import Function
from sympy.core.singleton import S
# from sympy.concrete.summations import Sum
from sympy.functions.elementary.exponential import log
from sympy.ntheory.factor_ import factorint, perfect_power, divisors
from sympy.ntheory.primetest import isprime
from sympy.ntheory.residue_ntheory import jacobi_symbol as number_jacobi, legendre_symbol as number_legendre
from sympy.core.logic import fuzzy_and, fuzzy_not

# n = Symbol('n', integer=True, positive=True)
# k = Symbol('k', integer=True, positive=True)
# p = Symbol('p', prime=True)

In [2]:
class primenu(Function):
    """little omega
    https://en.wikipedia.org/wiki/Prime_omega_function
    https://oeis.org/A001221
    1 -> 0
    2 -> 1
    """
    is_integer = True
    is_nonnegative = True
    
    @classmethod
    def eval(cls, n):
        if n.is_integer is False:
            raise TypeError("n should be an integer")
        if n.is_positive is False:
            raise ValueError("n should be a positive integer")
        if n.is_prime is True:
            return S.One
        if n is S.One:
            return S.Zero
        if n.is_Integer is True:
            return S(len(factorint(n)))

    def _eval_rewrite(self, rule, args, **hints):
        if rule == primenu:
            # primenu(n**a * m**b) => primenu(n*m)
            arg = []
            for n, e in (_.as_base_exp() for _ in Mul.make_args(args[0])):
                if n.is_integer is True and n.is_positive is True and \
                   e.is_integer is True and e.is_positive is True:
                    if n.is_Integer is True:
                        arg.append(Mul(*factorint(n).keys()))
                    else:
                        arg.append(n)
                else:
                    # すべてが正整数でない場合、書き換えは行わない。
                    # 例えば、primenu(4 * (1/4)) を primenu(2 * (1/4)) と書き換えるのはまずい
                    return
            return primenu(Mul(*arg))
    
    def _latex(self, printer, exp=None):
        if exp is None:
            return r'\omega\left( %s \right)' % printer._print(self.args[0])
        return r'\omega^{%s}\left( %s \right)' % (exp, printer._print(self.args[0]))

In [3]:
class primeomega(Function):
    """big omega
    """
    is_integer = True
    is_nonnegative = True
    
    @classmethod
    def eval(cls, n):
        if n.is_integer is False:
            raise TypeError("n should be an integer")
        if n.is_positive is False:
            raise ValueError("n should be a positive integer")
        if n.is_prime is True:
            return S.One
        if n is S.One:
            return S.Zero
        if n.is_Integer is True:
            return S(sum(factorint(n).values()))
        
    def _eval_rewrite(self, rule, args, **hints):
        if rule == Add:
            # primeomega(n**a*m**b) => a*primeomega(n) + b*primeomega(m)
            arg = []
            for n, e in (_.as_base_exp() for _ in Mul.make_args(args[0])):
                if n.is_integer is True and n.is_positive is True and \
                   e.is_integer is True and e.is_positive is True:
                    if n.is_Integer is True:
                        arg.append(primeomega(n))
                    else:
                        arg.append(e*primeomega(n))
                else:
                    # すべてが正整数でない場合、書き換えは行わない。
                    return
            return Add(*arg)
    
    def _latex(self, printer):
        return r'\Omega\left( %s \right)' % printer._print(self.args[0])

In [4]:
class liouville_function(Function):
    is_integer = True
    
    @classmethod
    def eval(cls, n):
        if n.is_integer is False:
            raise TypeError("n should be an integer")
        if n.is_positive is False:
            raise ValueError("n should be a positive integer")
        if n.is_prime is True:
            return S.NegativeOne
        if n is S.One:
            return S.One
        if n.is_Integer is True:
            return S.NegativeOne if sum(factorint(n).values()) % 2 else S.One
        
    def _eval_rewrite(self, rule, args, **hints):
        if rule == primeomega:
            return S.NegativeOne**primeomega(*args)
        if rule == liouville_function:
            # liouville_function(n * m**2) => liouville_function(n)
            arg = []
            for n, e in (_.as_base_exp() for _ in Mul.make_args(args[0])):
                if n.is_integer is True and n.is_positive is True and \
                   e.is_integer is True and e.is_positive is True:
                    if n.is_Integer is True:
                        for p, s in factorint(n).items():
                            s %= 2
                            if (s*e).is_odd is True:
                                arg.append(p)
                            elif (s*e).is_odd is None:
                                arg.append(p**(s*e))
                    else:
                        if e.is_even is True:
                            pass
                        elif e.is_odd is True:
                            arg.append(n)
                        else:
                            arg.append(n**e)
                else:
                    # すべてが正整数でない場合、書き換えは行わない。
                    return
            return liouville_function(Mul(*arg))
    
    def _latex(self, printer):
        return r'\lambda\left( %s \right)' % printer._print(self.args[0])

In [5]:
class mobius(Function):
    is_integer = True
    is_prime = False
    
    @classmethod
    def eval(cls, n):
        if n.is_integer is False:
            raise TypeError("n should be an integer")
        if n.is_positive is False:
            raise ValueError("n should be a positive integer")
        if n.is_prime is True:
            return S.NegativeOne
        if n is S.One:
            return S.One
        if n.is_Integer is True:
            factors = factorint(n)
            if any(e > 1 for e in factors.values()):
                return S.Zero
            return S.NegativeOne if len(factors) % 2 else S.One
        for p, e in (_.as_base_exp() for _ in Mul.make_args(n)):
            if p.is_Integer is True:
                if any(e > 1 for e in factorint(p).values()):
                    return S.Zero
            elif e > 1:
                return S.Zero
        
    def _latex(self, printer):
        return r'\mu\left( %s \right)' % printer._print(self.args[0])

In [6]:
class Legendre(Function):
    is_integer = True
    is_prime = False
    
    @classmethod
    def eval(cls, n, m):
        if n.is_integer is False:
            raise TypeError("n should be an integer")
        if m.is_integer is False:
            raise TypeError("m should be an integer")
        if m.is_prime is False or m.is_odd is False:
            raise ValueError("m should be an odd prime integer")
        if (n % m).is_zero is True:
            return S.Zero
        if n is S.One:
            return S.One
        if n is S.NegativeOne:
            return S.NegativeOne**((m - 1)/2)
        if n.is_Integer is True and m.is_Integer is True:
            return number_legendre(n, m)
        
    def _eval_rewrite(self, rule, args, **hints):
        if rule == Legendre:
            # low of quadratic reciprocity
            p, q = args
            if p.is_prime is True and p.is_odd is True and\
               q.is_prime is True and q.is_odd is True:
                return S.NegativeOne**((p-1)*(q-1)/4) / Legendre(q, p)
        if rule == Mul:
            n, p = args
            if p.is_prime is True and p.is_odd is True:
                arg = []
                for m, e in (_.as_base_exp() for _ in Mul.make_args(n)):
                    if m.is_integer is True and e.is_integer:
                        if m.is_Integer is True:
                            for s, k in factorint(m).items():
                                arg.append((s, k*e))
                        else:
                            arg.append((m, e))
                    else:
                        return
                return Mul(*[Legendre(m, p)**e for m, e in arg])

    def _latex(self, printer, exp=None):
        n, m = self.args
        n = printer._print(n)
        m = printer._print(m)
        if exp is None:
            return r'\left( \frac{%s}{%s} \right)' % (n, m)
        return r'\left( \frac{%s}{%s} \right)^{%s}' % (n, m, exp)

In [7]:
class Jacobi(Function):
    is_integer = True
    is_prime = False
    
    @classmethod
    def eval(cls, n, m):
        if n.is_integer is False:
            raise TypeError("n should be an integer")
        if m.is_integer is False:
            raise TypeError("m should be an integer")
        if m.is_positive is False or m.is_odd is False or m is S.One:
            raise TypeError("m should be a positive odd integer")
        if n is S.Zero:
            return S.Zero
        if n is S.One:
            return S.One
        if n is S.NegativeOne:
            return S.NegativeOne**((m - 1)/2)
        if n.is_Integer is True and m.is_Integer is True:
            return number_jacobi(n, m)
            
        
    def _latex(self, printer):
        n, m = self.args
        n = printer._print(n)
        m = printer._print(m)
        return r'\left( \frac{%s}{%s} \right)' % (n, m)

In [8]:
class von_mangoldt(Function):
    is_real = True
    is_negative = False
    
    @classmethod
    def eval(cls, n):
        if n.is_integer is False:
            raise TypeError("n should be an integer")
        if n.is_positive is False:
            raise TypeError("n should be a positive integer")
        if n.is_prime is True:
            return log(n)
        if n.is_Integer is True:
            power = perfect_power(n)
            if not power:
                return S.Zero
            p, e = power
            if not isprime(p):
                return S.Zero
            return log(p)

    def _latex(self, printer):
        return r'\Lambda\left( %s \right)' % printer._print(self.args[0])

In [9]:
class totient(Function):
    is_integer = True
    is_positive = True
    
    @classmethod
    def eval(cls, n):
        if n.is_integer is False:
            raise TypeError("n should be an integer")
        if n.is_positive is False:
            raise ValueError("n should be a positive integer")
        if n is S.One:
            return S.One
        if n.is_prime is True:
            return n - 1
        if n.is_Integer is True:
            t = 1
            for p, k in factorint(n).items():
                t *= p**(k - 1) * (p - 1)
            return S(t)

    def _latex(self, printer):
        return r'\varphi\left( %s \right)' % printer._print(self.args[0])

In [10]:
class reduced_totient(Function):
    """ A002322
    """
    is_integer = True
    is_positive = True
    
    @classmethod
    def eval(cls, n):
        if n.is_integer is False:
            raise TypeError("n should be an integer")
        if n.is_positive is False:
            raise ValueError("n should be a positive integer")
        if n.is_prime is True:
            return n - 1
        # 未実装

In [11]:
class divisor_sigma(Function):
    is_integer = True
    is_positive = True
    
    @classmethod
    def eval(cls, n, k=S.One):
        if n.is_integer is False:
            raise TypeError("n should be an integer")
        if n.is_positive is False:
            raise ValueError("n should be a positive integer")
        if k.is_integer is False:
            raise TypeError("k should be an integer")
        if k.is_nonnegative is False:
            raise ValueError("n should be a nonnegative integer")
        if n.is_prime is True:
            return 1 + n**k
        if n is S.One:
            return S.One
        if n.is_Integer is True:
            if k.is_zero is True:
                return Mul(*[e + 1 for e in factorint(n).values()])
            if k.is_Integer is True:
                return Mul(*[(p**(k*(e + 1)) - 1) // (p**k - 1) for p, e in factorint(n).items()])
            if k.is_zero is False:
                return Mul(*[cancel((p**(k*(e + 1)) - 1) / (p**k - 1)) for p, e in factorint(n).items()])

    def _latex(self, printer):
        if len(self.args) == 2:
            n, k = self.args
            n = printer._print(n)
            k = printer._print(k)
            return r'\sigma_{%s}\left( %s \right)' % (k, n)
        else:
            return r'\sigma\left( %s \right)' % printer._print(self.args[0])
        
class divisor_count(divisor_sigma):
    @classmethod
    def eval(cls, n):
        return super().eval(n, S.Zero)
    
    def _latex(self, printer):
        return r'\sigma_0\left( %s \right)' % printer._print(self.args[0])